In [1]:
#importing all those moduel that are required 
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [0]:
# Get images
image = img_to_array(load_img('woman.jpg'))
image = np.array(image, dtype=float)


In [22]:
#To get the shape of the image
print(image.shape)
print(image[:,:,0].shape)
print(image[:,:,1:].shape)

(400, 400, 3)
(400, 400)
(400, 400, 2)


The image consist of 3 layers(RGB) which is of 400x400 pixels

In [0]:
#We convert RGB colors to the Lab color space.
#Using grey scale layer as an input for neural network.
X = rgb2lab(1.0/255*image)[:,:,0]
#Two colored layer(two color layers, green–red and blue–yellow) will be used as an input.
Y = rgb2lab(1.0/255*image)[:,:,1:]
#dividing by 128, so to bound the range of ouput between -1 and 1.
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

1.0/255 indicates that we are using a 24-bit RGB color space. It means that we are using numbers between 0–255 for each color channel. This results in 16.7 million color combinations.

Since humans can only perceive 2–10 million colors, it does not make much sense to use a larger color space.

In [0]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, None, None, 8)     80        
_________________________________________________________________
conv2d_11 (Conv2D)           (None, None, None, 8)     584       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, None, None, 16)    1168      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, None, None, 16)    2320      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, None, None, 32)    4640      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, None, None, 32)    9248      
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, None, None, 32)   

The above give us the detail about our neural newtwork model and also the input and output dimension of each layer with total number of paramerter of the model.

In [0]:
model.compile(optimizer='rmsprop',loss='mse')



*   **rmsprop** :- RmsProp optimizer. RmsProp is an optimizer that utilizes the magnitude of recent gradients to normalize the gradients
*   **mse** :- meansquared error .



In [9]:
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=1000)

Epoch 1/1000
1/1 [==============================] - 1s 846ms/step - loss: 0.8477
Epoch 2/1000
1/1 [==============================] - 0s 310ms/step - loss: 0.7547
Epoch 3/1000
1/1 [==============================] - 0s 307ms/step - loss: 0.3988
Epoch 4/1000
1/1 [==============================] - 0s 318ms/step - loss: 0.2960
Epoch 5/1000
1/1 [==============================] - 0s 304ms/step - loss: 0.0175
Epoch 6/1000
1/1 [==============================] - 0s 303ms/step - loss: 0.0109
Epoch 7/1000
1/1 [==============================] - 0s 319ms/step - loss: 0.0088
Epoch 8/1000
1/1 [==============================] - 0s 303ms/step - loss: 0.0080
Epoch 9/1000
1/1 [==============================] - 0s 307ms/step - loss: 0.0074
Epoch 10/1000
1/1 [==============================] - 0s 306ms/step - loss: 0.0069
Epoch 11/1000
1/1 [==============================] - 0s 316ms/step - loss: 0.0066
Epoch 12/1000
1/1 [==============================] - 0s 301ms/step - loss: 0.0064
Epoch 13/1000
1/1 [======

In [10]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

1/1 [==============================] - 0s 105ms/step
0.00034087360836565495
